In [39]:
!pip install requests
!pip install huggingface_hub
!pip install pillow
!pip install feedparser
!pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [5]:
import feedparser

# Parse the RSS feed
rss_url = "https://buttondown.email/ainews/rss"
feed = feedparser.parse(rss_url)

# Extract titles and descriptions
news_items = [
    {"title": entry.title, "link": entry.link, "description": entry.description}
    for entry in feed.entries
]

# Pick the most interesting article (e.g., the latest one)
top_news = news_items[1] if news_items else None


In [22]:
# from bs4 import BeautifulSoup
# import html

# # Get the HTML content
# html_content = top_news["description"]

# # Parse and prettify the HTML
# soup = BeautifulSoup(html_content, 'html.parser')
# pretty_html = soup.prettify()

# # Display the prettified HTML
# print(pretty_html)

In [6]:
# Extract description and remove Reddit Recap section
desc = top_news["description"]
start_idx = desc.find("<h1 id=\"ai-reddit-recap\">")
end_idx = desc.find("<h1 id=\"ai-discord-recap\">")
if start_idx != -1 and end_idx != -1:
    desc = desc[start_idx:end_idx]
desc


'<h1 id="ai-reddit-recap">AI Reddit Recap</h1>\n<h2 id="rlocalllama-recap">/r/LocalLlama Recap</h2>\n<p><strong>Theme 1. Nvidia RTX 5090 enters production with 32GB VRAM</strong></p>\n<ul>\n<li><strong>Nvidia RTX 5090 with 32GB of RAM rumored to be entering production</strong> (<a href="https://reddit.com/r/LocalLLaMA/comments/1gqk300/nvidia_rtx_5090_with_32gb_of_ram_rumored_to_be/" target="_blank">Score: 271, Comments: 139</a>): <strong>Nvidia</strong> is reportedly shifting its production focus to the <strong>RTX 50 series</strong>, with the <strong>RTX 5090</strong> rumored to feature <strong>32GB of RAM</strong>. Concerns are rising about potential scalper activity affecting the availability and pricing of these new GPUs, as highlighted in multiple sources including <a href="https://videocardz.com/newz/nvidia-shifts-production-to-geforce-rtx-50-series-only-one-ada-gpu-reportedly-still-in-production" target="_blank">VideoCardz</a> and <a href="https://www.pcgamesn.com/nvidia/geforce

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")


In [ ]:

messages = [
	{
		"role": "user",
		"content": f"""From the following list of news items, pick the most interesting few and do a very short but appealing summary of it.
					{desc}"""
	}
]

# messages = [
# 	{
# 		"role": "user",
# 		"content": f"""
#         From the following list of news items, pick the most interesting one. 
#         Then, write a detailed, vivid description focusing on visual elements or themes that could inspire an AI image generation model.
#         Write it out all in one paragraph, no headings.
#         {desc}
#         """
# 	}
# ]

# messages = [
#     {
#         "role": "user",
#         "content": f"""
#         From the following list of AI news items, pick the single most interesting one. Name title of the article.
#         Then convert the news item into a prompt for an AI image generation model. Make it as vivid and detailed as possible.
#         The description must be no more than three sentences and strictly tied to the news content. Do not make it general.
#         Get creative with the prompt, but keep it focused on the 1 article.
#         Note that the image generator is not good at writing text in images.

#         AI News items:
#         {desc}
#         """
#     }
# ]

In [23]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=HUGGINGFACE_API_KEY)

messages = [
    {
        "role": "user",
        "content": f"""
        From the following list of AI news items, pick the single most interesting one and do an appealing catchy headline for it.

        AI News items:
        {desc}
        """
    }
]

stream = client.chat.completions.create(
    model="google/gemma-1.1-7b-it", 
	messages=messages, 
	max_tokens=500, # is for the output of the model
	stream=True,
    temperature=0.8
)

for chunk in stream:
    print(chunk.choices[0].delta.content, end="")

## **AI News: Gemini 1.5 Pro - New Vision Model Outperforms GPT-4 and OpenAI Models**

**Main Point:** A new AI vision model, **Gemini 1.5 Pro**, has topped the prestigious **LMSys leaderboard**, surpassing models from both Google and OpenAI. 

**Key Highlights:**

* The model exhibits remarkable capabilities in image generation and analysis, outperforming GPT-4 and other leading models in tests.
* Despite the lack of formal benchmarks and limited user feedback, the model's performance suggests significant potential for diverse applications.
* Concerns arise about the fairness of the LMSys leaderboard and the need for more reliable evaluation methods for AI models.

In [43]:
gemma_output = ""
for chunk in stream:
    gemma_output += chunk.choices[0].delta.content
gemma_output


"## Most Interesting AI News:\n\n**1. Nvidia RTX 5090 Entering Production with 32GB VRAM:** \n- Speculation surrounding the release of the RTX 5090 GPU heightened.\n- Rumor suggesting 32GB of RAM for this high-end graphics card raises concerns about potential scalping and pricing issues.\n\n\n**2. Undetectable ML Model Backdoors Using Digital Signatures:**\n- New research demonstrates the ability to create undetectable backdoors in ML models, raising concerns about the security and robustness of these models.\n\n\n**3. New CogVideoX-5B Open Source Text-to-Video Model Released:**\n- A new text-to-video model, CogVideoX-5b, has been released with improved multiresolution capabilities.\n- The model can be fine-tuned using LoRA on an NVIDIA RTX 4090 GPU.\n\n\n**4. Stack Overflow Traffic Plummets as AI Tools Rise:**\n- The popularity of AI coding tools has allegedly led to a significant traffic decline on Stack Overflow.\n- Concerns about the platform's hostile culture and outdated informat

In [41]:
import requests

API_URL = "https://api-inference.huggingface.co/models/black-forest-labs/FLUX.1-dev"
headers = {"Authorization": f"Bearer {HUGGINGFACE_API_KEY}"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.content

image_bytes = query({
	"inputs": gemma_output,
})
# You can access the image with PIL.Image for example
import io
from PIL import Image
image = Image.open(io.BytesIO(image_bytes))

from datetime import datetime

# Generate filename with current datetime
filename = datetime.now().strftime("%Y%m%d_%H%M%S") + ".png"

# Save to pictures folder
image.save(f"pictures/{filename}")



In [46]:
readme_template = f"""
# Today's AI News

![Todays Image](pictures/{filename})

{gemma_output}
"""

# Write the new content to README
with open('README.md', 'w') as f:
    f.write(readme_template)
